# Procesamiento mediante el módulo Isinet

En este notebook, se demostrará el uso del módulo para la generación del dataset y entrenamiento de una red.

Se utilizará como archivos originales (raw) a jupyter notebooks con información básica sobre deep learning.

El primer paso es importar el módulo Isinet y generar la estructura de directorios. Si se tiene al directorio /data_2 con los notebooks, la siguientes celdas moverán todos los notebooks al subdirectorio /data_2/raw y crearán un archivo banned.txt, a ser usado más adelante.

In [1]:
import isinet

attrs = isinet.Attributes("../data_2")

If your data_path is relative, it should be used from the current working directory: c:\Users\saoga\OneDrive\Escritorio\Repos\TPI-RNP-Palabras-Relacionadas-ISI\dev


In [2]:
attrs.init_data_dir()

A continuación, se instanciará un proceso de Isinet. Esta clase funciona de wrapper para las funciones necesarias para procesar el dataset y entrenar la red.

Se define, además, una función especial para convertir el contenido de un notebook jupyter a texto plano. Esta función convierte las celdas con markdown a texto, ignorando símbolos especiales y expresiones matemáticas.

In [2]:
process = isinet.Process(attrs)

In [4]:
import json
import re

def strip_markdown_syntax(text):
    # Remove headers: lines starting with #, keep the text after #
    text = re.sub(r'^\s{0,3}#{1,6}\s*(.*)', r'\1', text, flags=re.MULTILINE)

    # Remove emphasis: *text* or _text_ or **text** or __text__
    text = re.sub(r'(\*\*|__)(.*?)\1', r'\2', text)  # bold
    text = re.sub(r'(\*|_)(.*?)\1', r'\2', text)    # italic

    # Remove inline code: `code`
    text = re.sub(r'`([^`]*)`', r'\1', text)

    # Remove links but keep link text: [text](url)
    text = re.sub(r'\[([^\]]+)\]\([^)]+\)', r'\1', text)

    # Remove images syntax: ![alt](url) — remove whole thing
    text = re.sub(r'!\[.*?\]\(.*?\)', '', text)

    # Remove blockquotes: lines starting with >
    text = re.sub(r'^\s{0,3}>\s?', '', text, flags=re.MULTILINE)

    # Remove unordered list markers (-, *, +)
    text = re.sub(r'^\s*([-*+])\s+', '', text, flags=re.MULTILINE)

    # Remove ordered list markers (1., 2., etc.)
    text = re.sub(r'^\s*\d+\.\s+', '', text, flags=re.MULTILINE)

    # Remove horizontal rules (---, ***, ___)
    text = re.sub(r'^\s*((\*\s*){3,}|(-\s*){3,}|(_\s*){3,})\s*$', '', text, flags=re.MULTILINE)

    # Remove extra backslashes used for escaping markdown special chars
    text = re.sub(r'\\([\\`\*_{}\[\]()#+\-.!])', r'\1', text)

    # Remove math expressions ($...$ or $$...$$)
    text = re.sub(r'\$\$.*?\$\$', '', text, flags=re.DOTALL)
    text = re.sub(r'\$(?:[^$\\]|\\.)*?\$', '', text)

    # Remove leftover lines starting with !
    text = re.sub(r'^\s*!\s*.*$', '', text, flags=re.MULTILINE)

    # Clean multiple blank lines
    text = re.sub(r'\n\s*\n+', '\n\n', text).strip()

    return text

def ipynb_to_plain(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        notebook = json.load(f)

    markdown_cells = [
        strip_markdown_syntax(''.join(cell['source']))  # Each cell['source'] is a list of lines
        for cell in notebook.get('cells', [])
        if cell.get('cell_type') == 'markdown'
    ]

    ret = '\n\n'.join(markdown_cells)

    return ret

process.raw_to_plain(ipynb_to_plain)

Convirtiendo archivo: 1_Detección_de_Objetos.ipynb
Convirtiendo archivo: 1_Embeddings.ipynb
Convirtiendo archivo: 1_Redes_Convolucionales.ipynb
Convirtiendo archivo: 1_Transformers.ipynb
Convirtiendo archivo: 1_VisualizacionFeatures.ipynb
Convirtiendo archivo: 2_AnchorBoxes.ipynb
Convirtiendo archivo: 2_Entrenamiento_Word2Vec.ipynb
Convirtiendo archivo: 2_FineTuning.ipynb
Convirtiendo archivo: 2_Implementacion_Transformers.ipynb
Convirtiendo archivo: 3_SSD.ipynb
Convirtiendo archivo: 3_TransferenciaEstilos.ipynb
Convirtiendo archivo: BERT.ipynb
Convirtiendo archivo: CLIP_Imágenes_y_Lenguaje_Natural.ipynb
Convirtiendo archivo: Copia_de_Capas_Convolucionales.ipynb
Convirtiendo archivo: Deep_Learning_y_GPU.ipynb
Convirtiendo archivo: Evaluación_de_los_Modelos.ipynb
Convirtiendo archivo: GANs.ipynb
Convirtiendo archivo: MecanismosDeAtencion.ipynb
Convirtiendo archivo: ModelosDeDifusión.ipynb
Convirtiendo archivo: Modelos_Personalizados.ipynb
Convirtiendo archivo: NLPDataset.ipynb
Convirtie

A continuación, a se tokenizará los archivos con el texto plano.

Primero se debe definir una función para tokenizar, y que a su vez indique por cada token si se trata de un token limpio o no.

In [5]:
import spacy

esp = spacy.load("es_core_news_sm")

def tokenizer_esp_spacy(txt):
    def is_clean_token(token):
        return not (
            token.is_punct or
            token.is_space or
            token.is_stop or
            len(token) == 1)
    
    tokens = esp.tokenizer(txt)

    ret = []

    for token in tokens:
        ret.append(isinet.Token(token.text, is_clean_token(token)))

    return ret

process.tokenize(tokenizer_esp_spacy)

Tokenizando archivo: 1_Detección_de_Objetos.ipynb
Tokenizando archivo: 1_Embeddings.ipynb
Tokenizando archivo: 1_Redes_Convolucionales.ipynb
Tokenizando archivo: 1_Transformers.ipynb
Tokenizando archivo: 1_VisualizacionFeatures.ipynb
Tokenizando archivo: 2_AnchorBoxes.ipynb
Tokenizando archivo: 2_Entrenamiento_Word2Vec.ipynb
Tokenizando archivo: 2_FineTuning.ipynb
Tokenizando archivo: 2_Implementacion_Transformers.ipynb
Tokenizando archivo: 3_SSD.ipynb
Tokenizando archivo: 3_TransferenciaEstilos.ipynb
Tokenizando archivo: BERT.ipynb
Tokenizando archivo: CLIP_Imágenes_y_Lenguaje_Natural.ipynb
Tokenizando archivo: Copia_de_Capas_Convolucionales.ipynb
Tokenizando archivo: Deep_Learning_y_GPU.ipynb
Tokenizando archivo: Evaluación_de_los_Modelos.ipynb
Tokenizando archivo: GANs.ipynb
Tokenizando archivo: MecanismosDeAtencion.ipynb
Tokenizando archivo: ModelosDeDifusión.ipynb
Tokenizando archivo: Modelos_Personalizados.ipynb
Tokenizando archivo: NLPDataset.ipynb
Tokenizando archivo: Optimizac

Teniendo los tokens limpios, se generará los conceptos candidatos con las siguientes líneas:

In [3]:
window_size = 5

In [21]:
candidates = process.get_candidate_concepts(window_size)

Detectando conceptos en archivo: 1_Detección_de_Objetos.ipynb
Detectando conceptos en archivo: 1_Embeddings.ipynb
Detectando conceptos en archivo: 1_Redes_Convolucionales.ipynb
Detectando conceptos en archivo: 1_Transformers.ipynb
Detectando conceptos en archivo: 1_VisualizacionFeatures.ipynb
Detectando conceptos en archivo: 2_AnchorBoxes.ipynb
Detectando conceptos en archivo: 2_Entrenamiento_Word2Vec.ipynb
Detectando conceptos en archivo: 2_FineTuning.ipynb
Detectando conceptos en archivo: 2_Implementacion_Transformers.ipynb
Detectando conceptos en archivo: 3_SSD.ipynb
Detectando conceptos en archivo: 3_TransferenciaEstilos.ipynb
Detectando conceptos en archivo: BERT.ipynb
Detectando conceptos en archivo: CLIP_Imágenes_y_Lenguaje_Natural.ipynb
Detectando conceptos en archivo: Copia_de_Capas_Convolucionales.ipynb
Detectando conceptos en archivo: Deep_Learning_y_GPU.ipynb
Detectando conceptos en archivo: Evaluación_de_los_Modelos.ipynb
Detectando conceptos en archivo: GANs.ipynb
Detecta

Teniendo los conceptos candidatos, se experimenta para generar vocabularios con diversos rangos de frecuencias.

Para esto, se definió la función auxiliar `div_by_2`.

Tras prueba y error, se optó por utilizar el vocabulario con el ragno de frecuencias [(4.4,5.6), (3.3, 4.2), (2.2, 2.8), (1.1, 1.4)]

In [22]:
div_by_2 = lambda start_min, start_max: [(start_min*(4-i), start_max*(4-i)) for i in range(0,window_size-1)]

freq_ranges = [
    div_by_2(1.1,1.4)
]

process.make_vocabs(candidates, window_size, freq_ranges)

Generando vocabulario: 0 ([(4.4, 5.6), (3.3000000000000003, 4.199999999999999), (2.2, 2.8), (1.1, 1.4)])


Tras escribir ciertos tokens en banned.txt, se generaron nuevamente los conceptos candidatos y el vocabulario.

Es importante que el formato de rompimiento de línea del archivo banned.txt coincida con el establecido en Attributes.

Finalmente, se debe seleccionar el vocabulario creado:

In [3]:
vocab_len = process.select_vocabulary("vocab_0")

print(f"Longitud del vocabulario: {vocab_len}")

Longitud del vocabulario: 182


Teniendo seleccionado el vocabulario, se lo debe usar para tokenizar por conceptos los archivos con todos los tokens (no solo los limpios).

Se debe definir un factor de extensión de la ventana, para compensar por el agregado de los tokens no limpios respecto al momento en que se detectó los conceptos candidatos. Como la conversión de los archivos raw a plain es altamente confiable (no aparecen caracteres adicionales en lugares extraños), se usará un factor de 2 (menor al factor por defecto, 3).

In [5]:
window_size_extension_factor = 2

process.tokenize_by_concepts(window_size, window_size_extension_factor)

Tokenizando por conceptos: 1_Detección_de_Objetos.ipynb
Tokenizando por conceptos: 1_Embeddings.ipynb
Tokenizando por conceptos: 1_Redes_Convolucionales.ipynb
Tokenizando por conceptos: 1_Transformers.ipynb
Tokenizando por conceptos: 1_VisualizacionFeatures.ipynb
Tokenizando por conceptos: 2_AnchorBoxes.ipynb
Tokenizando por conceptos: 2_Entrenamiento_Word2Vec.ipynb
Tokenizando por conceptos: 2_FineTuning.ipynb
Tokenizando por conceptos: 2_Implementacion_Transformers.ipynb
Tokenizando por conceptos: 3_SSD.ipynb
Tokenizando por conceptos: 3_TransferenciaEstilos.ipynb
Tokenizando por conceptos: BERT.ipynb
Tokenizando por conceptos: CLIP_Imágenes_y_Lenguaje_Natural.ipynb
Tokenizando por conceptos: Copia_de_Capas_Convolucionales.ipynb
Tokenizando por conceptos: Deep_Learning_y_GPU.ipynb
Tokenizando por conceptos: Evaluación_de_los_Modelos.ipynb
Tokenizando por conceptos: GANs.ipynb
Tokenizando por conceptos: MecanismosDeAtencion.ipynb
Tokenizando por conceptos: ModelosDeDifusión.ipynb
Toke

Mediante una llamada a una función, se generarán múltiples datasets, variando el tamaño de la ventana.

Además, se debe especificar el número K, que indica cuántos ejemplos negativos habrá por cada positivo (es decir, concepto en el contexto).

In [5]:
concept_window_sizes = [25, 50, 75, 100, 150, 200]
K = 5

process.make_datasets(concept_window_sizes, K)

Armando dataset con: 1_Detección_de_Objetos.ipynb
Armando dataset con: 1_Embeddings.ipynb
Armando dataset con: 1_Redes_Convolucionales.ipynb
Armando dataset con: 1_Transformers.ipynb
Armando dataset con: 1_VisualizacionFeatures.ipynb
Armando dataset con: 2_AnchorBoxes.ipynb
Armando dataset con: 2_Entrenamiento_Word2Vec.ipynb
Armando dataset con: 2_FineTuning.ipynb
Armando dataset con: 2_Implementacion_Transformers.ipynb
Armando dataset con: 3_SSD.ipynb
Armando dataset con: 3_TransferenciaEstilos.ipynb
Armando dataset con: BERT.ipynb
Armando dataset con: CLIP_Imágenes_y_Lenguaje_Natural.ipynb
Armando dataset con: Copia_de_Capas_Convolucionales.ipynb
Armando dataset con: Deep_Learning_y_GPU.ipynb
Armando dataset con: Evaluación_de_los_Modelos.ipynb
Armando dataset con: GANs.ipynb
Armando dataset con: MecanismosDeAtencion.ipynb
Armando dataset con: ModelosDeDifusión.ipynb
Armando dataset con: Modelos_Personalizados.ipynb
Armando dataset con: NLPDataset.ipynb
Armando dataset con: Optimizac

Finalmente, se entrenarán múltiples modelos, variando los tamaños de embedding y de lote, por cada dataset generado.

In [5]:
lr = 0.002
num_epochs = 10
embed_sizes = [64, 128, 256, 512]
batch_sizes = [128, 256]

process.train_multiple(lr, num_epochs, embed_sizes, batch_sizes)

Entrenando modelo dataset-100-128-64
loss 0.692, 30285.8 tokens/sec on cpu
loss 0.688, 58441.1 tokens/sec on cpu
loss 0.683, 90470.4 tokens/sec on cpu
loss 0.675, 121924.1 tokens/sec on cpu
loss 0.662, 161907.9 tokens/sec on cpu
loss 0.645, 182426.0 tokens/sec on cpu
loss 0.625, 228346.5 tokens/sec on cpu
loss 0.604, 195405.8 tokens/sec on cpu
loss 0.583, 284657.5 tokens/sec on cpu
loss 0.566, 306399.1 tokens/sec on cpu
Entrenando modelo dataset-100-128-128
loss 0.692, 27191.5 tokens/sec on cpu
loss 0.685, 58174.6 tokens/sec on cpu
loss 0.676, 87467.4 tokens/sec on cpu
loss 0.662, 113152.9 tokens/sec on cpu
loss 0.641, 148692.8 tokens/sec on cpu
loss 0.615, 179837.1 tokens/sec on cpu
loss 0.588, 205376.4 tokens/sec on cpu
loss 0.564, 260584.9 tokens/sec on cpu
loss 0.543, 221846.1 tokens/sec on cpu
loss 0.526, 292068.4 tokens/sec on cpu
Entrenando modelo dataset-100-128-256
loss 0.690, 22035.6 tokens/sec on cpu
loss 0.678, 41920.1 tokens/sec on cpu
loss 0.660, 58593.3 tokens/sec on cpu

Habiendo entrenado estos modelos, solo resta probarlos. Esto puede hacerse mediante la función `get_related_concepts`.

En primer lugar, se debe cargar el modelo (con la clase SkipGram):

In [4]:
model_name = "dataset-25-256-512"

embed_size= int(model_name.split("-")[3])

model = isinet.SkipGram(process.vocabulary, embed_size)
model.load(attrs, model_name)

Algunos ejemplos de uso de la función para obtener los 10 conceptos más relacionados serían:

In [5]:
# ('datos', 'conjunto')
isinet.get_related_concepts(process.vocabulary, model, 8, 10)

[('práctica',),
 ('métodos',),
 ('datos', 'falsos'),
 ('residual',),
 ('memoria',),
 ('sintetizada',),
 ('métodos', 'aumento'),
 ('predecir', 'palabra'),
 ('propagación',),
 ('clase', 'etiqueta')]

In [6]:
# ('aprendizaje', 'supervisado')
isinet.get_related_concepts(process.vocabulary, model, 38, 10)

[('salida', 'capas'),
 ('veamos',),
 ('clase', 'etiqueta'),
 ('término', 'regularización'),
 ('complejidad',),
 ('algoritmo', 'aprendizaje'),
 ('red', 'profunda'),
 ('boxes', 'bounding'),
 ('paper',),
 ('entrada', 'convolución')]

In [7]:
# ('relu',)
isinet.get_related_concepts(process.vocabulary, model, 50, 10)

[('memoria',),
 ('stride',),
 ('centrales', 'palabras'),
 ('parametro', 'optimizar'),
 ('clase', 'etiqueta'),
 ('activación', 'sigmoidea'),
 ('principal',),
 ('calculado', 'parametro'),
 ('entrenar', 'red'),
 ('cpu', 'gpu')]

In [8]:
# ('activación', 'sigmoidea')
isinet.get_related_concepts(process.vocabulary, model, 55, 10)

[('relu',),
 ('atención', 'tokens'),
 ('entrada', 'original'),
 ('average', 'pooling'),
 ('semántica',),
 ('imagen', 'texto'),
 ('cabezales', 'atención'),
 ('optimización',),
 ('contexto', 'palabras'),
 ('acceso', 'parámetros')]

In [9]:
# ('traducción', 'automática')
isinet.get_related_concepts(process.vocabulary, model, 83, 10)

[('suma',),
 ('stride',),
 ('eficiencia', 'computacional'),
 ('entrada', 'altura'),
 ('datos', 'fashion-mnist'),
 ('machine', 'learning'),
 ('cabezales',),
 ('algoritmo', 'aprendizaje'),
 ('tensor', 'kernel'),
 ('entrenamos', 'modelo')]

In [10]:
# ('segmentación', 'instancias')
isinet.get_related_concepts(process.vocabulary, model, 176, 10)

[('atención', 'tokens'),
 ('práctica',),
 ('dropout', 'conexión'),
 ('funciona',),
 ('complejidad',),
 ('multiplicaciones', 'matrices'),
 ('clip', 'personalizado'),
 ('sintetizada',),
 ('padding',),
 ('implementamos', 'función')]